# Proyek Analisis Data: Nama dataset
- Nama: Muhamad Insan Rahmatuloh
- Email: muhamad.insan.r@gmail.com
- Id Dicoding:

## Menentukan Pertanyaan Bisnis

- dimana saja kota tempat tinggal customer dan kota mana yang paling banyak
- dari wilayah tsb dimana order paling banyak
- produk apa saja yang paling banyak diorder
- apa saja produk dengan skor review yang baik
- bagaimana presentase status order
- produk yang paling mahal 
- bagaimana analisis RFMnya
- bagaimana geoanalisis antara order, customer dan seller

## Menyaipkan semua library yang dibuthkan

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Data Wrangling

### Gathering Data 

In [2]:
customers = pd.read_csv('data/E-Commerce Public Dataset/customers_dataset.csv')
geolocation = gpd.read_file('data/E-Commerce Public Dataset/geolocation_dataset.csv')
geo_location = gpd.GeoDataFrame(geolocation, geometry=gpd.points_from_xy(geolocation['geolocation_lng'], geolocation['geolocation_lat']))
geo_location.crs = {'init':'epsg:4326'}
order_items = pd.read_csv('data/E-Commerce Public Dataset/order_items_dataset.csv')
order_payments = pd.read_csv('data/E-Commerce Public Dataset/order_payments_dataset.csv')
order_reviews = pd.read_csv('data/E-Commerce Public Dataset/order_reviews_dataset.csv')
orders = pd.read_csv('data/E-Commerce Public Dataset/orders_dataset.csv')
product_cat_name = pd.read_csv('data/E-Commerce Public Dataset/product_category_name_translation.csv')
products = pd.read_csv('data/E-Commerce Public Dataset/products_dataset.csv')
sellers = pd.read_csv('data/E-Commerce Public Dataset/sellers_dataset.csv')


d:\insan\Project-portfolio\docoding\latihan-wrangling\env-dico\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


### Assessing Data

#### Customers

In [3]:
customers.sample(10)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
11072,4a652b352b855e5e22de8d83490f14e7,17da2889e3f35318e13404eb3db8df4c,3470,sao paulo,SP
56168,428c9b0270df86e9cf60837fd129aa55,e57eb0d2ebc257996397e5ca8bec6572,83221,paranagua,PR
37372,97a0e2c05d5882609289a1816e234ba0,e983d6a5d6a92e29fcc48d7b42a6031e,13190,monte mor,SP
53297,c7041f9f3e88d79bf3c9f3bd1d9e6786,dfbf50eeb9fe8d91e69b973266cd8063,4937,sao paulo,SP
50926,92e9bc030ae552be2a2602a7a72e999b,883ce6de316525ee6c45bdc335c0494f,29101,vila velha,ES
99071,1612f32963e9825da1018976d1c9e8fc,0e2cdea5b5c9dc617ad5a66dea978812,40375,salvador,BA
85078,6231b6e4d05e6ab8e30b6fca4afff42b,bab25785bc0b036a7065bb1c5c5710de,71699,brasilia,DF
50291,18a0ae9ef526e6c02b8d1c384524e70e,f96fb3a4a96c2252110e0dd0240af56f,90245,porto alegre,RS
98860,75c12b4d4f984ae51c153fed33e1b4eb,966972c5fb82afe633c9e3d374cb9dbd,31720,belo horizonte,MG
50767,6a96055b35d60d2c4862552eaaa7dbb9,73d8f02720dd6554fd731b3db7973717,31160,belo horizonte,MG


In [4]:
customers.shape

(99441, 5)

In [5]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


#### Geolocation

In [6]:
geolocation.sample(5)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,geometry
37939,02865,-23.47676219401001,-46.67655711587511,sÃ£o paulo,SP,None
289525,13092,-22.892327093265806,-47.03560414524019,campinas,SP,None
573149,32210,-19.96305718649694,-44.0408842524263,contagem,MG,None
824636,78390,-15.069164244119936,-57.17361371772006,barra do bugres,MT,None
980411,96495,-31.45073006128839,-53.74481646169475,candiota,RS,None


In [7]:
geolocation.shape

(1000163, 6)

In [8]:
geolocation.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 6 columns):
 #   Column                       Non-Null Count    Dtype   
---  ------                       --------------    -----   
 0   geolocation_zip_code_prefix  1000163 non-null  object  
 1   geolocation_lat              1000163 non-null  object  
 2   geolocation_lng              1000163 non-null  object  
 3   geolocation_city             1000163 non-null  object  
 4   geolocation_state            1000163 non-null  object  
 5   geometry                     0 non-null        geometry
dtypes: geometry(1), object(5)
memory usage: 45.8+ MB


In [9]:
geolocation.geolocation_state.value_counts()

geolocation_state
SP    404268
MG    126336
RJ    121169
RS     61851
PR     57859
SC     38328
BA     36045
GO     20139
ES     16748
PE     16432
DF     12986
MT     12031
CE     11674
PA     10853
MS     10431
MA      7853
PB      5538
RN      5041
PI      4549
AL      4183
TO      3576
SE      3563
RO      3478
AM      2432
AC      1301
AP       853
RR       646
Name: count, dtype: int64

#### Order Items

In [10]:
order_items.sample(5)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
45721,67d413d255c5eb97d120a90d68e2c3d9,1,076e4ce53c8914382e7ff3732511b17e,59fb871bf6f4522a87ba567b42dafecf,2017-06-21 04:05:13,99.99,8.89
84705,c071755e071e5e8819f78265639b102e,1,5a848e4ab52fd5445cdc07aab1c40e48,c826c40d7b19f62a09e2d7c5e7295ee2,2018-01-22 20:57:20,122.99,15.61
96866,dbcb89699a4736bcef1373eb9d8ae5d1,1,cc5447118c174dcc6456c84ccb29e6f7,ef0ace09169ac090589d85746e3e036f,2018-08-07 23:30:11,117.40,18.92
87896,c78b306b7a03ebdabf44e1d351f83427,1,ace2ee0158ebad178e6616ad39507b8d,eed78ac17f7f795a19a709745f00cd4e,2018-01-23 12:30:03,59.90,12.76
8514,13707439a728b23bd3033dfab70b9840,1,e13f46cf68d356e0ac0f32058af20911,0ea22c1cfbdc755f86b9b54b39c16043,2018-02-06 02:55:30,24.90,21.15


In [11]:
order_items.shape

(112650, 7)

In [12]:
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [13]:
order_items.describe()

,order_item_id,price,freight_value
count,112650.000000,112650.000000,112650.000000
mean,1.197834,120.653739,19.990320
std,0.705124,183.633928,15.806405
min,1.000000,0.850000,0.000000
25%,1.000000,39.900000,13.080000
50%,1.000000,74.990000,16.260000
75%,1.000000,134.900000,21.150000
max,21.000000,6735.000000,409.680000


#### Order Payments

In [14]:
order_payments.sample(5)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
81220,28603c1c1a61da3897ea97a1846b1d22,1,debit_card,1,60.60
77539,b3ada39a1e9a71ee3b5dc5d61b32f0ca,1,credit_card,2,195.92
97844,20e7a21a233fcb32bbbb2d3613b03426,1,credit_card,7,176.99
101428,8ccf9f0aa61387140e1e58cff856f9e3,1,credit_card,2,270.85
5055,4d018030375b4b3ff3510d0c869c7b9e,1,credit_card,1,86.85


In [15]:
order_payments.shape

(103886, 5)

In [16]:
order_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [17]:
order_payments.describe()

,payment_sequential,payment_installments,payment_value
count,103886.000000,103886.000000,103886.000000
mean,1.092679,2.853349,154.100380
std,0.706584,2.687051,217.494064
min,1.000000,0.000000,0.000000
25%,1.000000,1.000000,56.790000
50%,1.000000,1.000000,100.000000
75%,1.000000,4.000000,171.837500
max,29.000000,24.000000,13664.080000


#### Orders Reviews

In [18]:
order_reviews.sample(5)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
79648,3e98d8e2fdf450bfbe94b7a08b8fde44,d8e4f17201e35b0341b90bf7b8be215c,3,NaN,NaN,2018-08-31 00:00:00,2018-09-03 12:30:17
74546,86c310ae4f7d8fea55914c81a4b02113,1cbf88b2aa2bdeff779e6afe29620c25,5,NaN,Muito bom,2018-02-27 00:00:00,2018-03-05 13:01:23
91908,171ab9025f2887a6650df6e1c0bd26b0,52fe284b8a7ca9585835524dbd80515b,5,NaN,NaN,2017-03-24 00:00:00,2017-03-25 17:40:44
6749,a38d355f8ea88f70ab7136b41afb04b5,666eb76141bb728bad22631162332455,5,NaN,NaN,2017-08-18 00:00:00,2017-08-19 19:01:17
41223,06eb17fd3e1c11929f0fdf971880193a,b9e60cfd587e2b40cf959a04023247a8,5,NaN,NaN,2018-02-10 00:00:00,2018-02-12 21:52:55


In [19]:
order_reviews.shape

(99224, 7)

In [20]:
order_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [21]:
order_reviews.describe()

,review_score
count,99224.000000
mean,4.086421
std,1.347579
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


#### Orders

In [22]:
orders.sample(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
28050,49c95c693647934576c041ce758a0310,34e9ab7101f384c10e91e5af8111bb3d,delivered,2017-08-17 16:09:34,2017-08-17 16:45:14,2017-08-18 15:25:45,2017-08-28 19:36:36,2017-09-06 00:00:00
36642,e1320973675a5fc37fab54cb22c582ab,b37ea7f1b8461e3bd4e534a81f126f5a,delivered,2018-07-16 16:56:04,2018-07-16 17:10:12,2018-08-01 12:00:00,2018-08-17 11:54:59,2018-08-07 00:00:00
69361,6f55fe858c5557f14d4b6ee5255d793b,55a736fdef3f978c92f0477297b92973,delivered,2017-04-12 20:56:03,2017-04-12 21:10:12,2017-04-24 10:35:41,2017-04-28 14:33:31,2017-05-08 00:00:00
86882,d0b446af99e79a960dd8c63777fc65df,9e63980f2ac755983af98be7a236a234,delivered,2017-10-21 20:21:29,2017-10-21 20:35:20,2017-10-24 21:04:18,2017-11-04 13:33:35,2017-11-03 00:00:00
55346,b22d306e18b3b665d49f2ce7d060742e,2abf58739c3d54ee365a1e45045d593e,delivered,2017-03-22 17:33:15,2017-03-22 17:33:15,2017-03-27 12:07:47,2017-03-31 12:27:17,2017-04-19 00:00:00


In [23]:
orders.shape

(99441, 8)

In [24]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [25]:
orders.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

#### Product Category Name Translation

In [26]:
product_cat_name.sample(5)

,product_category_name,product_category_name_english
40,casa_construcao,home_construction
29,pet_shop,pet_shop
6,perfumaria,perfumery
1,informatica_acessorios,computers_accessories
28,fashion_roupa_masculina,fashion_male_clothing


In [27]:
product_cat_name.shape

(71, 2)

In [28]:
product_cat_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


#### Products

In [29]:
products.sample(5)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
14230,779de9dd06ff30d5097a959a1e9dacdc,automotivo,59.0,1031.0,2.0,2250.0,61.0,22.0,22.0
3205,05216a143a65ab59fc48fb996e7c12d7,fashion_bolsas_e_acessorios,53.0,353.0,1.0,200.0,16.0,2.0,11.0
12238,488445f08d5b536d6aaa8dfe5c06a3db,beleza_saude,53.0,821.0,1.0,3500.0,35.0,15.0,15.0
11434,708281087dc97b0e50baa8891e741d8f,utilidades_domesticas,42.0,333.0,1.0,1000.0,21.0,23.0,20.0
21466,3e6d98f0757ce033a71509cf730ab7d6,ferramentas_jardim,41.0,442.0,6.0,200.0,16.0,6.0,11.0


In [30]:
products.shape

(32951, 9)

In [31]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [32]:
products.isnull().sum()

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [33]:
products.describe()

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32341.000000,32341.000000,32341.000000,32949.000000,32949.000000,32949.000000,32949.000000
mean,48.476949,771.495285,2.188986,2276.472488,30.815078,16.937661,23.196728
std,10.245741,635.115225,1.736766,4282.038731,16.914458,13.637554,12.079047
min,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,42.000000,339.000000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,51.000000,595.000000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,57.000000,972.000000,3.000000,1900.000000,38.000000,21.000000,30.000000
max,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000


#### Sellers

In [34]:
sellers.sample(5)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
1014,e3d3b3efac954a60d61be70b7e05b49b,3370,sao paulo,SP
261,f9903dd0f812c7e771fcad47b6fff231,89070,blumenau,SC
390,955fee9216a65b617aa5c0531780ce60,4782,sao paulo,SP
350,c42fd8e4d47dfb18ce5222f2dd7752f9,6246,osasco,SP
2833,4559697a8f7e637227c2eeaed843baff,19029,presidente prudente,SP


In [35]:
sellers.shape

(3095, 4)

In [36]:
sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


### Cleaning Data

In [37]:
# change data type on Order Items of shipping_limit_date to date
order_items.shipping_limit_date = pd.to_datetime(order_items['shipping_limit_date'].str.strip())

# change data type on order review of the review_creation_date and review_answer_timestamp to date 
order_reviews.review_creation_date = pd.to_datetime(order_reviews.review_creation_date.str.strip())
order_reviews.review_answer_timestamp = pd.to_datetime(order_reviews.review_answer_timestamp.str.strip())

# change data type on Orders of the order_purchase_timestamp, order_approved_at, order_delivered_carrier_date, 
# order_delivered_customer_date, order_estimated_delivery_date to date 
orders.order_purchase_timestamp = pd.to_datetime(orders.order_purchase_timestamp.str.strip())
orders.order_approved_at = pd.to_datetime(orders.order_approved_at.str.strip())
orders.order_delivered_carrier_date = pd.to_datetime(orders.order_delivered_carrier_date.str.strip())
orders.order_delivered_customer_date = pd.to_datetime(orders.order_delivered_customer_date.str.strip())
orders.order_estimated_delivery_date = pd.to_datetime(orders.order_estimated_delivery_date.str.strip())


In [38]:
# cleaning null value on Orders dataset (99441, 8)
# orders.isnull().sum()
# orders.dropna(subset=['order_approved_at','order_delivered_carrier_date','order_delivered_customer_date'], inplace=True)

# cleaning null value on Products dataset (32951, 9)
# products.isnull().sum()
# products.dropna(inplace=True)

In [39]:
print(f'order {orders.shape} & products {products.shape}')

order (99441, 8) & products (32951, 9)


## Exploratory Data Analysis (EDA)

### Explore ...

In [63]:
customers[customers.customer_state == 'CE']

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
59,a905baa530258422594f1b05615bd225,c80da60feddb7cf8325bd104032e314a,60140,fortaleza,CE
70,4c7241af24b5344cb01fe687643de4fe,b157c176c3fe04914fde33f2dc8b878a,60336,fortaleza,CE
103,126cf4e9fd189d2d64bc9b112fd6a5e8,90d479448b39ad9586d05da64cfb695b,62016,sobral,CE
303,3aec4688b93d6061ec5061721ad493a7,7b0cae22ae2b6cf88163a50600da4ee2,60181,fortaleza,CE
422,522a6e7c1f1f992ac81be6abefa46294,282ebddb3e85120b1652204758fc332b,60055,fortaleza,CE
...,...,...,...,...,...
99072,18e56af97c2f24afcfaf7aa97ad2b969,d9ff7e37d7bf448abb783a6e56462d93,63765,sucesso,CE
99171,3218724bc18c442e58bd39608b0f100a,d9151698fc7193669598995a4c6f494f,62800,aracati,CE
99328,1e03a2aa5714007700c8e727849cf123,54a2d9cbeb08a11268085529cfe2524f,60440,fortaleza,CE
99343,9289d8bcd95805bc1ac27c949a99da98,ec8aaeb3869352e348097f26f4c49ee3,60864,fortaleza,CE


In [72]:
custby_city_states = customers.groupby(by=['customer_state','customer_city']).customer_id.nunique().sort_values(ascending=False).to_frame().reset_index()
custby_city_states.sample(3)

,customer_state,customer_city,customer_id
799,SP,pinhalzinho,12
880,SP,charqueada,10
2247,MG,sao sebastiao do maranhao,3


In [73]:
order_items.sample(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
110271,fa8b91d8aacb92e77cd50059ac39c474,1,c4baedd846ed09b85f78a781b522f126,a1043bafd471dff536d0c462352beb48,2017-12-04 12:52:28,89.99,50.50
44029,6413a8cd79f9720426fd12968b388228,3,d016a1a4d0898c213be8443b0c99c784,30829ded4523ab9224b93bc49a62c95f,2018-07-05 10:53:51,199.00,23.32
102878,e9a1f1bac2a48afae0dbe612a3aa684b,1,ace5d86cf1ac63cdb76f49e5cd23d2f8,cca3071e3e9bb7d12640c9fbe2301306,2017-03-17 15:10:18,50.90,17.20


In [ ]:
orders.sample(2)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
20336,c2411b9c7822d227e3e48018405a0594,464ad1fa19f2a649e39193e5c98b8300,delivered,2018-01-25 23:24:56,2018-01-25 23:35:37,2018-01-26 20:19:09,2018-02-02 21:19:12,2018-02-09
17791,ad829c3ec314b40679b5cf05f1f27025,d0186eb06aa53361d8406e9f209a11b9,delivered,2018-01-09 11:21:48,2018-01-09 11:33:04,2018-01-16 00:57:00,2018-02-17 13:09:09,2018-02-08


In [78]:
orders_items = pd.merge(left=orders, right=order_items, on='order_id', how='inner')
orders_items['total_price'] = orders_items.order_item_id * orders_items.price
orders_items.sample(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,total_price
88146,ee8f5d7599c575d8f8eb1a2ea7d66686,ce86321953a0e590bef64f5823dac5fd,delivered,2017-09-05 08:40:16,2017-09-05 08:50:21,2017-09-11 21:33:21,2017-09-21 15:22:28,2017-09-25,1,82c3ea364e5480f335f6743ee725a921,70a12e78e608ac31179aea7f8422044b,2017-09-13 08:50:21,25.00,15.10,25.00
30610,69499ad287db14f2ca4ccf16a67536b0,a8abfe715d37e28718e35fc36acdb5d6,delivered,2018-06-19 12:05:40,2018-06-19 17:45:43,2018-06-20 14:43:00,2018-06-22 18:06:33,2018-07-04,1,bef82b9c3489736794c29f6d4a686ca6,9646c3513289980f17226a2fc4720dbd,2018-06-25 17:45:43,28.00,8.89,28.00
45503,7ef259be178f9f06c72a57add85b71cd,a16a13abd16a9d85e4b382ecdc192d33,delivered,2018-06-06 16:55:17,2018-06-06 17:13:19,2018-06-29 15:22:00,2018-07-12 20:03:51,2018-08-09,1,f4f4debbcfcafe6858d1e37a1f6e436e,5058e8c1e82653974541e83690655b4a,2018-06-28 17:13:19,139.98,232.69,139.98
104006,fecda210da4a36546b91948ee43af90d,66af94f0d6175f030731ae8ae7ed580c,delivered,2018-03-29 16:42:35,2018-03-29 16:55:28,2018-04-04 02:20:57,2018-05-02 20:36:49,2018-04-23,1,be4d4aacf88585bba5689cbc3393e0cc,50c9975695009e5e6473912e83a6d1da,2018-04-04 16:55:28,17.50,27.08,17.50
76319,452045e853d5fa6f87a6ebed979b3134,37ffb68c39bd78aec200de254446972f,delivered,2017-10-07 17:14:13,2017-10-07 17:28:18,2017-10-09 22:12:47,2017-10-18 18:19:59,2017-11-08,3,422879e10f46682990de24d770e7f83d,1f50f920176fa81dab994f9023523100,2017-10-13 17:28:18,59.90,17.67,179.70


In [86]:
orders_items_payments = pd.merge(left=orders_items, right=order_payments, on='order_id')
print(orders_items_payments.shape)
orders_items_payments.sample(3)
# world.continent.value_counts()

(117601, 19)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,total_price,payment_sequential,payment_type,payment_installments,payment_value
79537,61fc33807aeed929a157ce216496e6d1,418df4354874776f10838d1b905a3f65,delivered,2017-05-07 14:31:46,2017-05-07 14:41:58,2017-05-09 10:39:33,2017-05-15 09:49:59,2017-06-05,1,ef30fdd053284aaffa42dfd5612b66d8,066a6914e1ebf3ea95a216c73a986b91,2017-05-16 14:41:58,379.00,17.40,379.00,1,credit_card,10,396.40
58691,6b8a860cec63279fe49b5649ca6bab42,81d7e215787f9556b381145e2ac7d9f2,delivered,2017-09-10 12:11:43,2017-09-12 04:23:54,2017-09-15 21:39:56,2017-09-26 16:29:13,2017-10-02,1,b1287ed5e48382f6d6e5e1c561ac3738,d354c38a7182125a748cb10474fe963b,2017-09-18 04:23:54,69.99,15.24,69.99,1,boleto,1,85.23
29967,4d9f516707478b41c5932a3a3fcfb733,a9d950ed5c897a58ed2214e0d90b0356,delivered,2017-03-14 11:05:20,2017-03-14 11:05:20,2017-03-16 06:43:33,2017-03-23 09:52:35,2017-04-11,1,7c1bd920dbdf22470b68bde975dd3ccf,cc419e0650a3c5ba77189a1882b7556a,2017-03-23 11:05:20,64.99,16.15,64.99,1,boleto,1,81.14


In [92]:
all_orders = pd.merge(left=orders_items_payments, right=order_reviews, on='order_id')
all_orders.sample(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
59904,c2ebe69807422446d5e8edb2060da5e9,63f9f01b26b742afbd95beb988883c52,delivered,2018-03-05 03:46:04,2018-03-06 03:50:56,2018-03-08 22:18:31,2018-04-04 19:47:59,2018-04-17,1,0a04ebe637717bf7f0a9290adb51971f,...,1,boleto,1,200.05,ef68cb63e9831882c77c30cc0de9858d,2,NaN,NaN,2018-04-05,2018-04-18 14:39:23
21656,861d85972797c770a5139774b13efc48,4bd28e750151e0fcc738533e973a01b8,delivered,2017-07-26 16:44:38,2017-07-27 11:43:26,2017-07-27 18:51:54,2017-08-04 20:22:34,2017-08-21,1,4520766ec412348b8d4caa5e8a18c464,...,1,boleto,1,55.10,8ff95067fcb029ce2cea8a57a93f5f77,4,NaN,Entrega rápida e produto perfeito.,2017-08-05,2017-08-08 13:52:42
84594,ddaacb72e21c68c2863a84aa4b3e281d,0a00fd16d3a68e73856892a6a30201f8,delivered,2017-10-20 15:37:43,2017-10-20 17:28:31,2017-10-23 17:27:39,2017-10-24 20:28:56,2017-11-01,2,56eb39314ca59c39efa1e28beb073abe,...,1,credit_card,2,60.36,b558dc55f741b1a5ed3c2b47b67e8049,5,NaN,"Lindas, amei, veio certinho o meu produtos.",2017-10-25,2017-10-25 17:24:32


In [94]:
print(products.shape)
products.sample(3)

(32951, 9)


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
24353,9545d45c37449ccbc376de3a04c66e71,beleza_saude,39.0,1749.0,1.0,1350.0,27.0,9.0,18.0
10906,dea0538113c6e4b96b4d34f5ad453075,informatica_acessorios,59.0,569.0,1.0,5275.0,45.0,27.0,31.0
26015,a177ae30ef17394975b58f6d5883a56f,pet_shop,47.0,725.0,6.0,14150.0,45.0,20.0,35.0


In [95]:
print(product_cat_name.shape)
product_cat_name.sample(3)

(71, 2)


,product_category_name,product_category_name_english
20,audio,audio
14,brinquedos,toys
64,cds_dvds_musicais,cds_dvds_musicals


In [97]:
products_cat_name_translation = pd.merge(left=products, right=product_cat_name, on='product_category_name')
print(products_cat_name_translation.shape)
products_cat_name_translation.sample(3)

(32328, 10)


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
2630,3e67af30c5e3eae465fbf496aa88fc8b,esporte_lazer,46.0,1352.0,3.0,450.0,28.0,13.0,24.0,sports_leisure
9469,c7d729c7966df13084efeade5576dfef,moveis_decoracao,58.0,779.0,3.0,9450.0,16.0,80.0,50.0,furniture_decor
24788,b1ffef38f0db9fb4e1a6f91b47ec63d4,telefonia,45.0,636.0,2.0,100.0,20.0,20.0,20.0,telephony


#### Merger customer, order and product

In [100]:
order_cust = pd.merge(left=all_orders, right=customers, on='customer_id')
all_dataset = pd.merge(left=order_cust, right=products_cat_name_translation, on='product_id')
print(all_dataset.shape)
all_dataset.sample(5)

(115609, 38)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,customer_state,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
109531,59c0ed646a3b30d4054298988188486f,8ba643878d111b600a1a781ae8ec0983,delivered,2017-11-20 20:41:26,2017-11-20 20:56:22,2017-11-21 19:51:40,2017-11-30 19:53:08,2017-12-11,2,99c88c191bbed92f5be4ca3b00eb1a42,...,RJ,cama_mesa_banho,55.0,615.0,1.0,450.0,16.0,10.0,16.0,bed_bath_table
43136,366fb8cd92296c6541132ab4cbb28bac,841841043bf76dbb83f79c7a36dcf24b,delivered,2017-08-25 10:04:15,2017-08-25 10:15:19,2017-08-28 16:13:18,2017-08-30 16:03:37,2017-09-18,1,bdef57e26e989447aaf5ae0f7ccadf23,...,SP,esporte_lazer,34.0,733.0,1.0,233.0,17.0,15.0,16.0,sports_leisure
58344,f12388e0e8c7694fc38c8de19261054c,de481cb60487d1171ef850f3d818467c,delivered,2018-02-24 18:03:19,2018-02-26 17:30:05,2018-02-28 13:05:51,2018-04-18 13:08:46,2018-03-23,1,afc6bc70dc56fcf15c7f9f1e4bc67dda,...,RJ,utilidades_domesticas,59.0,303.0,1.0,9900.0,61.0,31.0,31.0,housewares
86857,08869d221628341d81a9dd18a07f625a,a48d8fbdbed3a8b9840c3f6bb5215a86,delivered,2018-05-06 22:31:38,2018-05-08 19:35:27,2018-05-10 15:09:00,2018-05-18 20:12:41,2018-05-23,1,0983cd4a5cabf1099659ce461511963c,...,MG,beleza_saude,58.0,1307.0,1.0,350.0,35.0,16.0,20.0,health_beauty
6815,d486475824974ef6fb73375f5a50e465,2dbc6c19eaeb265c5bed88607eb98f21,delivered,2017-06-23 22:04:36,2017-06-23 22:15:19,2017-06-28 12:15:28,2017-07-04 07:34:35,2017-07-19,1,283dc451ad3918badb976d56ff887289,...,ES,cama_mesa_banho,58.0,188.0,1.0,1650.0,44.0,2.0,35.0,bed_bath_table


## Visualization & Explanatory Analysis

### Pertanyaan 1: dimana saja kota tempat tinggal customer dan kota mana yang paling banyak, dari wilayah tsb dimana order paling banyak

### Pertanyaan 2: produk apa saja yang paling banyak diorder

### Pertanyaan 3: apa saja produk dengan skor review yang baik

### Pertanyaan 4: bagaimana presentase status order

### Pertanyaan 5: produk yang paling mahal

### Pertanyaan 6: bagaimana analisis RFMnya

### Pertanyaan 7: bagaimana geoanalisis antara order, customer dan seller

## Conclusion

- Conclution pertanyaan 1
- conclution pertanyaan 2